# Target Description

For our Target we wanted to find the number of Sunday church attendees for each tract. We want to filter out any non-Church buildings for this and filter out any non-Meeting house buildings such as temples. Although Churches have visitors that don't attend that meeting house we believed that these numbers don't need to be filtered out because these visitors avg out over time.

# How It Was Calculated

First we filter out all non church buildings in that places table using filter rlike. second we join this filtered table with patters leaving on the patterns of the church buildings. Third we will grab total visits and divide by  sunday attendance  for the popularity by day column to get the %of attendance that was on sunday. we will then explode the visitor_home_aggregation column and multiply the value by %of attendance that was on sunday. we will then group by tract code summing all the values in the value column. This should give us an idea of how many people go to church in each tract.

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import lower,col,udf
from pyspark.sql.functions import *
import pyspark.sql.functions as F

# load database
#%run ./build_database

# get safegraph data
places = spark.table("safegraph.places")
patterns = spark.table("safegraph.patterns")
censusblock_table = spark.table("safegraph.censusblock_table")
tract_table = spark.table("safegraph.tract_table")

lds_church = places.filter(
    (F.col("top_category") == "Religious Organizations") &
    (F.col("location_name").rlike("Latter|latter|Saints|saints|LDS|\b[Ww]ard\b")) &
    (F.col("location_name").rlike("^((?!Reorganized).)*$")) &
    (F.col("location_name").rlike("^((?!All Saints).)*$")) &
    (F.col("location_name").rlike("^((?![cC]ath).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]ody).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]eter).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]atholic).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]res).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]inist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ission).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]orship).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ain).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]aptist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]eth).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]vang).)*$")) &
    (F.col("location_name").rlike("^((?![Ll]utheran).)*$")) &
    (F.col("location_name").rlike("^((?![Oo]rthodox).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]piscopal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]abernacle).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]arvest).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ssem).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]edia).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artha).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]hristian).)*$")) &
    (F.col("location_name").rlike("^((?![Uu]nited).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]esl).)*$")) &
    (F.col("location_name").rlike("^((?![C]cosmas).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]reater).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]rison).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]ommuni).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]lement).)*$")) &
    (F.col("location_name").rlike("^((?![Vv]iridian).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]iocese).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]istory).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]chool).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hougt).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]oliness).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artyr).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ames).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]ouse).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]lory).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]nglican).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]oetic).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]anctuary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]quipping).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ohn).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ndrew).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]manuel).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]edeemed).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]erfecting).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ngel).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]rchangel).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]icheal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hought).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]ariosse).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]osmas).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]eliverance).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ociete).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]emple).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]eminary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]mployment).)*$")) &
    (F.col("location_name").rlike("^((?![Ii]nstitute).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]amp).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]tudent).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]ducation).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ocial).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]welfare).)*$")) &
    (F.col("location_name").rlike("^((?![Cc][Ee][Ss]).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]amily).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ary).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ussian).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]eautif).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]eaven).)*$")) &    
    (F.col("location_name").rlike("^((?!Inc).)*$")) &
    (F.col("location_name").rlike("^((?!God).)*$"))
  )

  # Join palces with patterns to work with the vis counts of each church
finding_churchs = patterns.join(other = lds_church, on = "placekey", how= "inner" )

finding_churchs = (finding_churchs.withColumn("Visits_per_visitor_avg", F.col("raw_visit_counts") / F.col("raw_visitor_counts"))
                   .withColumn("Sunday_visits", F.col("popularity_by_day")["Sunday"]).withColumn("sun_visit_%", F.col("Sunday_visits") / F.col("raw_visit_counts"))
                    )

# sum up all visitors for a tract from all the churchs 
finding_visit_count = finding_churchs.select("*", explode("visitor_home_aggregation"))
finding_visit_count = (finding_visit_count.withColumn("sunday_visit_count", F.col("value") * F.col("sun_visit_%"))
                       .groupBy("key").agg(F.sum("value"), F.avg("Visits_per_visitor_avg"))
                       )
finding_visit_count = (finding_visit_count.withColumnRenamed("key", "tractcode")
                       .withColumnRenamed("sum(value)", "visit_count_total")
                       .withColumnRenamed("avg(Visits_per_visitor_avg)", "avg_visits_per_visitor")
)

finding_visit_count = finding_visit_count.join(other = tract_table, on= "tractcode", how= "inner")

finding_visit_count = finding_visit_count.orderBy("tractcode")

# Diagram

The diagram gives a quick look at the key pieces of information we used to build the feature for our analysis. It shows the names of the different categories of data (like 'tractcode' for specific areas and ‘visit_count_total’ for the number of visits) and what type of information each category holds (like numbers or text). This summary helps everyone understand what the data is about and how it’s organized, making it simpler to work with and understand for further study.

In [ ]:
# Define the column names, data types, and descriptions
columns_data = [
    ("tractcode", "string", "Tract code"),
    ("visit_count_total", "bigint", "Total visit count"),
    ("avg_visits_per_visitor", "double", "Average visits per visitor"),
    ("state", "string", "State"),
    ("county", "string", "County"),
    ("tract", "string", "Tract"),
    ("name", "string", "Name"),
    ("land_m", "double", "Land area (in square meters)"),
    ("water_m", "double", "Water area (in square meters)"),
    ("lat", "double", "Latitude"),
    ("long", "double", "Longitude"),
    ("cnamelong", "string", "Long city name"),
    ("tractname", "string", "Tract name"),
    ("city", "string", "City"),
    ("county_name", "string", "County name"),
    ("city_population_2010", "int", "City population in 2010"),
    ("city_long", "double", "City longitude"),
    ("city_lat", "double", "City latitude")
]
# Create an HTML table with column names, data types, and descriptions
html_table = "<table><tr><th>Column Name</th><th>Data Type</th><th>Description</th></tr>"
for column_data in columns_data:
    html_table += f"<tr><td>{column_data[0]}</td><td>{column_data[1]}</td><td>{column_data[2]}</td></tr>"
html_table += "</table>"
# Display the HTML table
displayHTML(html_table)

Column Name,Data Type,Description
tractcode,string,Tract code
visit_count_total,bigint,Total visit count
avg_visits_per_visitor,double,Average visits per visitor
state,string,State
county,string,County
tract,string,Tract
name,string,Name
land_m,double,Land area (in square meters)
water_m,double,Water area (in square meters)
lat,double,Latitude


# What the Table Looks Like

This is how the 5 first rows of our final table look like.

In [ ]:
display(finding_visit_count.limit(5))

tractcode,visit_count_total,avg_visits_per_visitor,state,county,tract,name,land_m,water_m,lat,long,cnamelong,tractname,city,county_name,city_population_2010,city_long,city_lat,stusab,state_name,statens
16001000100,163,2.3069695424150987,16,001,000100,1,4141830.0,165711.0,43.6159696,-116.2075223,Ada County,1,Boise,Ada,205671,-116.2310843,43.5984881,ID,Idaho,01779783
16001000201,398,2.7218009641502747,16,001,000201,2.01,1.4234202E7,18273.0,43.6314866,-116.1710011,Ada County,2.01,Boise,Ada,205671,-116.2310843,43.5984881,ID,Idaho,01779783
16001000202,906,2.60431052114343,16,001,000202,2.02,3.3914385E7,51222.0,43.6751115,-116.2170641,Ada County,2.02,Garden City,Ada,10972,-116.293701,43.6694085,ID,Idaho,01779783
16001000302,185,2.490006561392791,16,001,000302,3.02,2041614.0,9686.0,43.6518985,-116.2435264,Ada County,3.02,Garden City,Ada,10972,-116.293701,43.6694085,ID,Idaho,01779783
16001000303,67,2.5041206575477384,16,001,000303,3.03,1798904.0,224344.0,43.6589164,-116.2692899,Ada County,3.03,Garden City,Ada,10972,-116.293701,43.6694085,ID,Idaho,01779783


# Visualization

[See Image](my_image.png)